# Appendix
https://en.wikipedia.org/wiki/2048_(video_game) 
https://www.geeksforgeeks.org/2048-game-in-python/

problem statement : https://drive.google.com/file/d/1-l9t-sKYWD_hrAAmfzJFmv2X151lMsEo/view?usp=drive_link

IMPLEMENTED GAME
4X4 GRID (implemented as a numpy grid)
4 switches to move all up/all down /all right /all left
(moves with the keyboard up/l/d/r buttons)


# game user manual 
i desgned first the basic version of the game with help of the given regulations 
then i added the required moves counter to show the number of moves taken


# game

In [10]:
import pygame
import random
from math import log
import numpy as np

In [11]:
# --- Constants ---
GRID_SIZE = 4
TILE_SIZE = 100
MARGIN = 10
WIDTH = (TILE_SIZE + MARGIN) * GRID_SIZE
HEIGHT = (TILE_SIZE + MARGIN) * GRID_SIZE + 50 #Added 50 for moves display
FPS = 30

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

TILE_COLORS = {
    x: ((255 - 15*(log(x)/log(2))),255,(255 - 15*(log(x)/log(2)))) for x in [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
}
TILE_COLORS[0] = GRAY  # Gray for empty tiles


In [ ]:
class Game2048:
    def __init__(self):
        self.size = GRID_SIZE

        self.board = np.zeros((self.size, self.size), dtype=int)    # initialize the board

        self.moves = 0                          # initialize moves 

        self.add_new_tile()                     # Add the first tile
        self.add_new_tile()                     # Add the second tile
        self.game_over = False
        self.has_won = False

    '''This function is used to add a new tile (2 or 4) to the board at a random empty position
    The tile is 2 with 90% probability and 4 with 10% probability'''

    def add_new_tile(self):
        empty_cells = np.argwhere(self.board == 0)
        if empty_cells.size > 0:
            i, j = random.choice(empty_cells)
            self.board[i, j] = 2 if random.random() < 0.9 else 4

    def transpose(self):
        # Transpose the board (used for vertical moves)
        self.board = np.transpose(self.board)

    def reverse(self):
        # Reverse each row of the board (used for right moves)
        for i in range(self.size):
            self.board[i] = np.flip(self.board[i])

    def slide(self, row):
        # Slide all non-zero tiles to the left
        row = row[row != 0]
        return np.pad(row, (0, self.size - len(row)), 'constant')

    def merge(self, row):
        # Merge adjacent tiles with the same value
        row = row.copy()
        i = 0
        while i < self.size - 1:
            if row[i] == row[i + 1] and row[i] != 0:
                row[i] *= 2
                row[i + 1] = 0
                i += 1  # Skip the next tile since it was merged
            i += 1
        return row

    # IMPLEMENTED LOGIC :

    '''The logic is whenever the two tiles of same number is adjacent (after eliminating empty slides from between ) would be merged into the one , on the side we have moved
    now after merging the tiles , there is again vacant space in between , so we again remove the empty tiles but not merge it again '''

    def move_left(self):
        moved = False
        for i in range(self.size):
            original_row = self.board[i].copy()
            row = self.slide(original_row)
            row = self.merge(row)
            row = self.slide(row)

            if not np.array_equal(row, original_row):
                moved = True
            self.board[i] = row
        return moved

    def move_right(self):                           # similar to what we did in left but on reverse of the row
        # Perform a right move
        self.reverse()
        moved = self.move_left()
        self.reverse()
        return moved

    def move_up(self):                              # similar to what we did in left but on transpose of left
        # Perform an up move
        self.transpose()
        moved = self.move_left()
        self.transpose()
        return moved

    def move_down(self):                            # similar to what we did in left but on reverse of transpose
        self.transpose()
        moved = self.move_right()
        self.transpose()
        return moved

    # when there are no merges and moves possible we have game over
    def is_game_over(self):                            
        # Check if no moves are possible
        if np.any(self.board == 0):
            return False 

        for i in range(self.size):
            for j in range(self.size):
                if i < self.size - 1 and self.board[i][j] == self.board[i + 1][j]:
                    return False 
                if j < self.size - 1 and self.board[i][j] == self.board[i][j + 1]:
                    return False 

        return True # Game over

    '''when any of the tile reaches 2048 '''
    def check_win(self):
        # Check if the player has reached 2048
        return np.any(self.board == 2048)

    '''how do you react to any input from the user'''
    '''when the input clicks up , then the move up function and similarly for other directions'''

    def input(self, key):
        # Handle user input and update the game state
        old_board = self.board.copy()

        if key == pygame.K_UP:
            moved = self.move_up()
        elif key == pygame.K_DOWN:
            moved = self.move_down()
        elif key == pygame.K_LEFT:
            moved = self.move_left()
        elif key == pygame.K_RIGHT:
            moved = self.move_right()

        if not np.array_equal(old_board, self.board):
            self.moves += 1
            self.add_new_tile()
            if self.is_game_over():
                self.game_over = True
            if self.check_win():
                self.has_won = True


In [13]:
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2048")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 40)
large_font = pygame.font.Font(None, 60)

In [ ]:
game = Game2048()
running = True
while running:
    clock.tick(FPS)

    #input interaction
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            game.input(event.key)

    screen.fill(WHITE)                                                                              

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            value = game.board[i][j]
            color = TILE_COLORS.get(value, GRAY)                                                    
            rect = pygame.Rect(j * (TILE_SIZE + MARGIN) + MARGIN,
                               i * (TILE_SIZE + MARGIN) + MARGIN,
                               TILE_SIZE, TILE_SIZE)
            pygame.draw.rect(screen, color, rect)

            if value != 0:
                text = font.render(str(value), True, BLACK)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)

    #Draw Moves
    moves_text = font.render(f"Moves: {game.moves}", True, BLACK)
    moves_rect = moves_text.get_rect(center=(WIDTH // 2, HEIGHT - 25)) #Position below the grid
    screen.blit(moves_text, moves_rect)

    #Game Over message
    if game.game_over:
      game_over_text = font.render("Game Over!", True, BLACK)
      game_over_rect = game_over_text.get_rect(center = (WIDTH//2, HEIGHT//2))
      screen.blit(game_over_text,game_over_rect)

    if game.has_won:
        win_text = large_font.render("You Win!", True, BLACK)
        win_rect = win_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(win_text, win_rect)

    pygame.display.flip() 

pygame.quit()